In [3]:
import numpy as np
import plotly.graph_objects as go
import csv

In [10]:
data = []
with open('../data/0_31G.csv') as f:
    reader = csv.reader(f)

    for column in reader:
        column[0] = column[0].replace('(', '').replace("'" , '')
        column[2] = column[2].replace(')', '').replace("'" , '')
        column[1] = column[1].replace("'" , '')
        column[0] = float(column[0])
        column[1] = float(column[1])
        column[2] = float(column[2])
        data.append(column)
        # print(data)
    data = np.array(data)
    print(data)

[[ -17.5   -7.5 -490. ]
 [  12.5   17.5 -490. ]
 [  -2.5   17.5 -490. ]
 [   2.5   22.5 -490. ]
 [  17.5    7.5 -490. ]
 [ -12.5   12.5 -490. ]
 [   2.5   17.5 -490. ]
 [  17.5  -12.5 -490. ]
 [  22.5    2.5 -490. ]
 [ -17.5   -2.5 -490. ]
 [   2.5   17.5 -490. ]
 [  -7.5   17.5 -490. ]
 [   2.5  -17.5 -490. ]
 [   7.5  -17.5 -490. ]
 [  17.5   -2.5 -490. ]
 [  -7.5  -17.5 -490. ]
 [  -7.5  -17.5 -490. ]
 [ -12.5  -12.5 -490. ]
 [  -7.5   17.5 -490. ]
 [  -2.5   17.5 -490. ]
 [   7.5  -12.5 -490. ]
 [ -12.5   12.5 -490. ]
 [   2.5    2.5 -490. ]
 [   2.5    2.5 -490. ]]


In [38]:

fig = go.Figure(data=go.Scatter(x=data[:,0], y=data[:,2], mode='markers'))
fig.show()


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed